<a href="https://colab.research.google.com/github/barroso2501/GIS_MapBiomas/blob/main/Mosaic_mapbiomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!apt-get install -qq libgdal-dev -y
!pip install -q GDAL


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Para Marina executar
#fazer o mosaico a partir de 1994 até 2021
import os
import glob
from osgeo import gdal

# Defina o caminho para os arquivos tif em seu Google Drive
path = '/content/drive/MyDrive/MAPBIOMAS-EXPORT'

# Pegue todos os arquivos que atendem ao padrão mencionado - AQUI ATUALIZA ANO
file_list = glob.glob(os.path.join(path, '/content/drive/MyDrive/MAPBIOMAS-EXPORT/mapbiomas_-*.tif'))

if not file_list:
    print("Nenhum arquivo correspondente encontrado. Verifique o caminho e o padrão.")
    exit()

# Supondo que todos os arquivos tenham 38 bandas como você mencionou
num_bands = 1

# Comece o loop a partir da banda 11
start_band = 1

for i in range(start_band, num_bands + 1):
    temp_files = []

    # Extraia cada banda para arquivos temporários
    for file in file_list:
        dst_file = os.path.join(path, f"temp_band_{i}_{os.path.basename(file)}")
        temp_files.append(dst_file)

        ds = gdal.Open(file)
        band = ds.GetRasterBand(i)

        array_data = band.ReadAsArray()

        # Verificação adicional
        if array_data is None or len(array_data.shape) != 2:
            print(f"Erro ao ler a banda {i} do arquivo {file}. Pulando esta banda.")
            continue

        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create(dst_file, band.XSize, band.YSize, 1, gdal.GDT_Byte, options=['NBITS=4'])
        out_ds.GetRasterBand(1).WriteArray(array_data)
        out_ds.SetProjection(ds.GetProjection())
        out_ds.SetGeoTransform(ds.GetGeoTransform())
        out_ds.FlushCache()
        out_ds = None
        ds = None

    # Mosaico dos arquivos temporários - ATUALIZAR ANO AQUI
    output_file = os.path.join(path, f"amz_pasto{i}.tif")
    cmd = ['gdal_merge.py', '-o', output_file, '-co', 'NBITS=4'] + temp_files
    result = os.system(' '.join(cmd))

    # Remover arquivos temporários
    for temp_file in temp_files:
        os.remove(temp_file)

    # Verifique o status do comando
    if result == 0:
        print(f"Mosaico da banda {i} criado com sucesso!")
    else:
        print(f"Erro ao criar o mosaico da banda {i}.")


Mosaico da banda 1 criado com sucesso!


In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 55.3 MB/s eta 0:00:00


In [ ]:
import os
import rasterio
from rasterio.windows import Window
import numpy as np
from google.colab import drive

# Montar o Google Drive no ponto padrão
drive.mount('/content/drive')

def process_rasters_in_blocks(base_path="/content/drive/MyDrive/MAPBIOMAS-EXPORT",
                              prefix='mosaic_usos',
                              max_number=21,
                              block_size=(1024, 1024)):
    """
    Processa uma sequência de rasters em blocos e cria um novo raster.

    Args:
    base_path (str): Caminho base onde os rasters estão armazenados.
    prefix (str): Prefixo dos arquivos raster.
    max_number (int): Número máximo na sequência de rasters.
    block_size (tuple): Tamanho do bloco para processamento (altura, largura).

    Returns:
    np.array: Uma matriz numpy representando o raster final.
    """

    final_raster = None

    for i in range(1, max_number + 1):
        raster_path = os.path.join(base_path, f"{prefix}_{i}.tif")

        if not os.path.exists(raster_path):
            print(f"Arquivo não encontrado: {raster_path}")
            continue

        with rasterio.open(raster_path) as src:
            # Inicializar o raster final com as dimensões e tipo do primeiro raster
            if final_raster is None:
                final_raster = np.zeros((src.height, src.width), dtype=src.dtypes[0])

            # Processamento em blocos
            for ji, window in src.block_windows(1):
                raster_data = src.read(1, window=window)

                # Aplicar a lógica condicional
                value_to_assign = 22 - i
                condition = (raster_data == 1) & (final_raster[window.row_off:window.row_off+window.height,
                                                            window.col_off:window.col_off+window.width] == 0)
                final_raster[window.row_off:window.row_off+window.height,
                             window.col_off:window.col_off+window.width][condition] = value_to_assign

    return final_raster

base_path = '/content/drive/MyDrive/mapbiomas_brasil_col71_soy'
prefix = 'mosaic_soy'
max_number = 21

final_raster = process_rasters_in_blocks(base_path, prefix, max_number)

# Salvar o raster final
final_path = os.path.join(base_path, "sequenci_soy_1.tif")
with rasterio.open(final_path, 'w', driver='GTiff',
                   height=final_raster.shape[0], width=final_raster.shape[1],
                   count=1, dtype=final_raster.dtype) as dst:
    dst.write(final_raster, 1)

print(f"Raster final salvo em: {final_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


Raster final salvo em: /content/drive/MyDrive/mapbiomas_brasil_col71_soy/sequenci_soy_1.tif


In [ ]:
import os
import glob
from osgeo import gdal

# Defina o caminho para os arquivos tif em seu Google Drive
path = "/content/drive/MyDrive/Download/MAPBIOMAS-EXPORT"

# Pegue todos os arquivos que atendem ao padrão mencionado
file_list = glob.glob(os.path.join(path, "/content/drive/MyDrive/MAPBIOMAS-EXPORT/mapbiomas_*.tif"))

if not file_list:
    print("Nenhum arquivo correspondente encontrado. Verifique o caminho e o padrão.")
    exit()

# Supondo que todos os arquivos tenham 38 bandas como você mencionou
num_bands = 1

# Comece o loop a partir da banda 11
start_band = 1

# ...

for i in range(start_band, num_bands + 1):
    temp_files = []

    # Extraia cada banda para arquivos temporários
    for file in file_list:
        dst_file = os.path.join(path, f"temp_band_{i}_{os.path.basename(file)}")
        temp_files.append(dst_file)

        ds = gdal.Open(file)

        # Verifique se a banda existe
        if i > ds.RasterCount:
            print(f"O arquivo {file} não tem banda {i}. Pulando este arquivo.")
            continue

        band = ds.GetRasterBand(i)

        array_data = band.ReadAsArray()

        # Verificação adicional
        if array_data is None or len(array_data.shape) != 2:
            print(f"Erro ao ler a banda {i} do arquivo {file}. Pulando esta banda.")
            continue

        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create(dst_file, band.XSize, band.YSize, 1, gdal.GDT_Byte, options=['NBITS=2'])
        out_ds.GetRasterBand(1).WriteArray(array_data)
        out_ds.SetProjection(ds.GetProjection())
        out_ds.SetGeoTransform(ds.GetGeoTransform())
        out_ds.FlushCache()
        out_ds = None
        ds = None

    # ...


    # Mosaico dos arquivos temporários
    output_file = os.path.join(path, f"mosaic_LULC_{i}.tif")
    cmd = ['gdal_merge.py', '-o', output_file, '-co', 'NBITS=4'] + temp_files
    result = os.system(' '.join(cmd))

    # Remover arquivos temporários
    for temp_file in temp_files:
        os.remove(temp_file)

    # Verifique o status do comando
    if result == 0:
        print(f"Mosaico da banda {i} criado com sucesso!")
    else:
        print(f"Erro ao criar o mosaico da banda {i}.")


Mosaico da banda 1 criado com sucesso!
